In [ ]:
# 거리 계산
def latilong(df, my_location):
    dat = []
    data_set = []
    for ty in range(len(df)):
        dat.append((df['stationLatitude'].iloc[ty],df['stationLongitude'].iloc[ty]))
        
    for rt in dat:
        sc = (float(rt[0]),float(rt[1]))
        data_set.append(haversine(my_location, sc, unit = 'm'))
    df_ty = pd.DataFrame(data_set, columns = ['distance'])
    return df_ty

# 최적의 따릉이 대여소
def bestdistance(df_ty,s):
    distance_list = list(df_ty['distance'].sort_values()[:3].index)
    df_total = s.loc[distance_list,['rackTotCnt','stationName','parkingBikeTotCnt','shared','stationLatitude','stationLongitude']].reset_index()
    df_total = pd.merge(df_total, df_ty['distance'].sort_values()[:3].reset_index(), on='index')
    return df_total

In [1]:
import pandas as pd
import requests
from haversine import haversine

In [2]:
df = pd.read_csv('data1.csv')

In [3]:
df

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId
0,15,102. 망원역 1번출구 앞,3,20,37.555649,126.910629,ST-4
1,14,103. 망원역 2번출구 앞,1,7,37.554951,126.910835,ST-5
2,13,104. 합정역 1번출구 앞,17,131,37.550629,126.914986,ST-6
3,5,105. 합정역 5번출구 앞,7,140,37.550007,126.914825,ST-7
4,12,106. 합정역 7번출구 앞,6,50,37.548645,126.912827,ST-8
...,...,...,...,...,...,...,...
2701,8,5864.장훈고 앞,5,63,37.511570,126.913979,ST-3200
2702,10,5865.더리브오피스텔 앞,0,0,37.517918,126.912598,ST-3204
2703,8,5866.포레나 당산,1,13,37.521954,126.895508,ST-3212
2704,5,6053. 중부세무서 앞,1,20,37.560925,126.990662,ST-3161


## 거리계산

In [4]:
def test(data_location, my_location):
    # print(my_location)
    bike_location = (data_location['stationLatitude'], data_location['stationLongitude'])
    distance = haversine(my_location, bike_location, unit = 'm')
    return distance

In [10]:
#값을 받아와야하는 부분
#주소: 서울 마포구 동교동
my_location = (37.55564880,126.91062927)

In [11]:
df[['stationLatitude','stationLongitude']] = df[['stationLatitude','stationLongitude']].astype(float)

In [12]:
df['distance'] = df[['stationLatitude','stationLongitude']].apply(test, axis =1, my_location = my_location)

In [33]:
distance_index = list(df['distance'].sort_values().index)[:5]

In [36]:
df.loc[distance_index]

,rackTotCnt,stationName,parkingBikeTotCnt,shared,stationLatitude,stationLongitude,stationId,distance
0,15,102. 망원역 1번출구 앞,3,20,37.555649,126.910629,ST-4,0.000000
1,14,103. 망원역 2번출구 앞,1,7,37.554951,126.910835,ST-5,79.719948
71,9,184. 7번가피자 서교망원점,1,11,37.558949,126.907753,ST-342,445.991463
326,25,497.합정동주민센터 앞,20,80,37.551754,126.912300,ST-2156,457.443985
2,13,104. 합정역 1번출구 앞,17,131,37.550629,126.914986,ST-6,677.558789


In [ ]:
#bike
#data1234

## 주변 맛집계산

In [ ]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [ ]:
driver.get('https://m.place.naver.com/place/list?query=%ED%95%A0%EB%A8%B8%EB%8B%88%EA%B0%80%EB%9E%98%EB%96%A1%EB%B3%B6%EC%9D%B4&level=top')
time.sleep(5)
re = driver.find_elements(By.CSS_SELECTOR,'.place_bluelink')
hg = driver.find_elements(By.CSS_SELECTOR,"#_list_scroll_container > div > div > div:nth-child(2) > ul > li> div.qbGlu > div.ouxiq.icT4K > a:nth-child(1)")
time.sleep(2)

store_name = []
for i,bb in zip(re,hg):
    ret = []
    ret.append(i.text)
    ret.append(bb.get_attribute('href')[-20:-10])
    store_name.append(ret)
menuNm = []
store_name = store_name[:10]
for li in store_name:
    code = str(li[-1])
    url = f'https://m.store.naver.com/restaurants/{code}/tabs/menus/baemin/list?more=false&entry=ple'
    driver.get(url)
    time.sleep(5)
    tk = driver.find_elements(By.CSS_SELECTOR, '.list_tit')

    for i in tk[2:7]:
        i.click()
        time.sleep(0.5)

    fd = driver.find_elements(By.CSS_SELECTOR, '.name')
    price = driver.find_elements(By.CSS_SELECTOR, '.price')

    for menu, mepr in zip(fd,price):
        temp = []
        temp.append(str(li[0]))
        temp.append(menu.text)
        temp.append(mepr.text)
        menuNm.append(temp)